# (FD22-0493) 聊天機械人的技術原理和應用


1. 瞭解人工智能發展的現況
2. 學習NLP自然語言的理論
3. 認識聊天機械人的技術和應用範圍
4. 利用Python語言實作簡單的聊天機械人




# 1.下載教材


In [ ]:
!git clone https://github.com/pcmsricky/chatbot
%cd chatbot

Cloning into 'chatbot'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 46 (delta 1), reused 8 (delta 1), pack-reused 36
Unpacking objects: 100% (46/46), done.
/content/chatbot


# 2. Python 中的變數

In [ ]:
#單獨的變數
num = 3           #數字變數
text = 'Hello'    #文字變數

print(num)

3


In [ ]:
#list, 一個變數可以儲存多個資料
nlist = [1,3,5]
nlist.append(8)

tlist = ['hello','morning']
xlist = ['hello',1,4,'Peter']

print(nlist)

[1, 3, 5, 8]


# 3. 在list中搜尋

In [ ]:
DB = [1,4,5,8]
target = 4

for x in DB:
  if x==target:
    print('I found it.')
    break
else:
  print('not found.')

I found it.


# 4. 用list做語料庫

In [ ]:
DB = ['hello','morning','sunday','What is your name?']
target = input('youAsk:')
for q in DB:
  if q==target:
    print('I found it.')
    break
else:
  print('not found.')    

youAsk:Sunday
not found.


# 5.在list中搜尋最接近的值, 神奇的tuple

In [ ]:
DB = [1,4,5,8]
target = 4

tmp = []
for x in DB:
  tmp.append(  (abs(target-x),x) )

print(tmp)  
DB = sorted(tmp)
print(DB)
diff,ans = DB[0]
print(ans)

[(3, 1), (0, 4), (1, 5), (4, 8)]
[(0, 4), (1, 5), (3, 1), (4, 8)]
4


# 6. 比較兩個句子中相同的字數

In [1]:
s1 = 'What is your name?'
s2 = 'what is this?'

list1 = s1.split()
print(list1)

list2 = s2.split()
print(list2)

count = 0
for a in list1:
  if a in list2:
    count = count+1
print(count)    


['What', 'is', 'your', 'name?']
['what', 'is', 'this?']
1


# 7. 自動回答機械人 ver#1

In [ ]:
DB = [('What is your name?','I am ricky.'),
      ('How old are you?','I am 18 years old.'),
      ('When does the class start?','The class starts at 8:05'),
      ('Where is puiching middle school?', 'Avenida de Horta e Costa, No.7, Macao.')
      ]

def match(alist,blist):
    count = 0
    for a in alist:
        if a in blist:
            count = count+1
    return count 

while (True):
    youAsk = input('youAsk: ')
    
    if youAsk=='bye':
        break
    else:
        askList = youAsk.split()
        nlist = []
        for q,a in DB:
            qList = q.split()
            c = match(askList,qList)
            nlist.append((c,a))
              
        nlist = sorted(nlist,reverse=True)
        c,a = nlist[0]
        print('bot:',a)

youAsk: name
bot: The class starts at 8:05
youAsk: name?
bot: I am ricky.
youAsk: bye


## 安裝 jieba 中文斷字工具

In [ ]:
!pip install jieba

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## 測試中文斷字

In [ ]:
import jieba
import logging
jieba.setLogLevel(logging.INFO)

askList = '你的名字是甚麼?'.split()
print(askList)

['你的名字是甚麼?']


# 8. 中文版的自動回答機械人 ver#2

In [ ]:
DB = [('你好嗎?', '我很好.'),
      ('你吃晚飯了嗎?', '吃了.'),
      ('你今年幾多歲?', '我今年18歲.'),
      ('你喜歡吃什麼?', '巧克力.')
      ]
	  

In [ ]:
import jieba
import logging
jieba.setLogLevel(logging.INFO)


def match(alist,blist):
    count = 0
    for a in alist:
        if a in blist:
            count = count+1
    return count 

def printDB():
    print('DB=[')
    for i in range(len(DB)-1):
        print(f'{DB[i]},')
    print(DB[len(DB)-1])    
    print(']') 

while (True):
    youAsk = input('你問: ')
    
    if youAsk=='再見':
        printDB()
        break
    else:
        askList = jieba.lcut(youAsk)
        nlist = []
        for q,a in DB:
            qList = jieba.lcut(q)
            c = match(askList,qList)
            nlist.append((c,a))
              
        nlist = sorted(nlist,reverse=True)
        c,a = nlist[0]
        if (c==0):
            new_answer=input('資料庫找不到對應的回答, 請教我如何回答:')
            DB.append((youAsk,new_answer))
        else:    
            print('機械人:',a)

你問: 早晨
資料庫找不到對應的回答, 請教我如何回答:hi
你問: 再見
DB=[
('你好嗎?', '我很好.'),
('你吃晚飯了嗎?', '吃了.'),
('你今年幾多歲?', '我今年18歲.'),
('你喜歡吃什麼?', '巧克力.'),
('早晨', 'hi')
]


In [ ]:
run cmd

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
    PID TTY      STAT   TIME COMMAND
      1 ?        Ss     0:00 /sbin/docker-init -- /datalab/run.sh
      8 ?        Sl     0:01 /tools/node/bin/node /datalab/web/app.js
     19 ?        Ss     0:00 tail -n +0 -F /root/.config/Google/DriveFS/Logs/dpb
     30 ?        Z      0:05 [python3] <defunct>
     31 ?        S      0:00 python3 /usr/local/bin/colab-fileshim.py
     44 ?        Sl     0:01 /usr/bin/python3 /usr/local/bin/jupyter-notebook --
     45 ?        Sl     0:00 /usr/local/bin/dap_multiplexer --domain_socket_path
     62 ?        Ssl    0:03 /usr/bin/python3 -m ipykernel_launcher -f /root/.lo
     82 ?        Sl     0:00 /usr/bin/python3 /usr/local/lib/python3.7/dist-pack
    158 ?        Sl     0:07 node /datalab/web/pyright/pyright-langserver.js --s
    189 ?        Sl     0:00 ./ngrok http 8000 --log=stdout
    193 ?        R      0:00 ps -ax


# 9. 中文版的自動回答機械人 ver#3

In [ ]:
with open("DB.txt", "r") as f:
    dataDB = f.read().rstrip()

DB = eval(dataDB)    

In [ ]:
from flask import Flask, request, send_from_directory
from ipywidgets import widgets
from IPython.display import display
import io
import jieba, json
from werkzeug.serving import run_simple



def match(alist,blist):
    cout = 0
    for a in alist:
        if a in blist:
            cout = cout+1
    return cout 

def printDB():
    print('DB=[')
    for i in range(len(DB)-1):
        print(f'{DB[i]},')
    print(DB[len(DB)-1])    
    print(']') 

app = Flask("AI chat")


@app.route("/")
def form():
    with open('chatbot.html', 'r',encoding='utf8') as chatbotfile:
        data = chatbotfile.read()
    return data

    
@app.route("/ask", methods=['POST', 'GET'])
def ask():
    if not request.args.get('text'):
        return '都沒有文字:<';
    
    youAsk = request.args.get('text')
    print('有人問: %s'%youAsk)
    
    askList = jieba.lcut(youAsk)
    nlist = []
    for q,a in DB:
        qList = jieba.lcut(q)
        c = match(askList,qList)
        nlist.append((c,a))

    nlist = sorted(nlist,reverse=True)
    c,a = nlist[0]
    if (c==0):
        new_answer=input('資料庫找不到對應的回答, 請教我如何回答:')
        DB.append((youAsk,new_answer))
        response = new_answer
    else:    
        response = a
   
    print('機械人:',response)
    return response



if __name__ == "__main__":
    run_simple('localhost', 8000, app)

 * Running on http://localhost:8000/ (Press CTRL+C to quit)
127.0.0.1 - - [24/May/2022 03:24:12] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/May/2022 03:24:19] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/May/2022 03:24:20] "GET /static/chatbot.ico HTTP/1.1" 200 -
127.0.0.1 - - [24/May/2022 03:25:12] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/May/2022 03:25:14] "GET /static/chatbot.ico HTTP/1.1" 200 -


有人問: Hi


# 10.自動回答的改進

## 「的的了了」不必計算

In [ ]:
dropList = ['的','了']
youAsk = '你吃晚飯了嗎?'
askList = jieba.lcut(youAsk)
#askList = [x for x in askList if x not in dropList]
print(askList)

['你', '吃', '晚飯', '了', '嗎', '?']


## 設定最低的匹配值


In [ ]:
nlist = [(2,'a'),(1,'b'),(0,'c')]
therehold = 3
nlist = [x for x in nlist if x[0]>therehold]
nlist = sorted(nlist,reverse=True)
if len(nlist)==0 or nlist[0][0]<0:
  print('not found')
else:
  print(nlist[0])

not found


## 挑戰圖靈測試

In [ ]:
import random
nlist = [(2,'a'),(2,'b'),(2,'c'),(1,'d')]
nlist = sorted(nlist,reverse=True)
maxi = nlist[0][0]
nlist = [x for x in nlist if x[0]==maxi]
random.shuffle(nlist)
print(nlist[0])

(2, 'b')


## 上下文處理

設定主謂賓語 或 時間、地點、人物等欄位, 下文未有提供的訊息, 用上文的補上

## 分佈式處理

設定多個主題自動回答機機械人, 利用分佈式網絡協同處理

## 將問題交比網絡自動回答機械人

In [ ]:
import requests
s = '你好嗎?'
resp = requests.get("http://api.qingyunke.com/api.php", {'key': 'free', 'appid':0, 'msg': s})
resp.encoding = 'utf8'
resp = resp.json()
print('菲菲：', resp['content'])


菲菲： 我不好，你都不愛我，ToT
